# CNN in TF
- Get EfficientNet 
    - inject general constructor
    - 

In [ ]:
!git clone https://github.com/qubvel/efficientnet.git libs/efficientnet && \
    sed -i '6i\EfficientNet   = inject_tfkeras_modules(model.EfficientNet)' libs/efficientnet/efficientnet/tfkeras.py && \
    sed -i '364 s/require_flatten=include_top/require_flatten=True/' libs/efficientnet/efficientnet/model.py

- calculate base $\phi$ for EFN

In [ ]:
import numpy as np
import glob
import cv2

import constants as c

total_res = np.array([0,0,0])
for img in glob.glob(f'{c.rois_sorted_dir}/*/agriculture/*.jpg'):
    total_res += cv2.imread(img).shape
    
print("Average resolution: ", total_res/len(glob.glob(f'{c.rois_sorted_dir}/*/agriculture/*.jpg')))
len(glob.glob(f'{c.rois_sorted_dir}/*/agriculture/*.jpg'))

- create model builder depending on parametric $\phi$ (see `./efn.py`)

---

In [ ]:
import os

import efn
from libs.efficientnet.efficientnet.tfkeras import preprocess_input
     
from metrics import f1, recall, precision

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.losses import binary_crossentropy

from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from keras_contrib.callbacks.cyclical_learning_rate import CyclicLR

import matplotlib.pyplot as plt
import io

- load data with data_generator

In [ ]:
def load_data(img_shape, data_path):
    datagenerator = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        horizontal_flip=True,
        vertical_flip=True)

    return datagenerator.flow_from_directory(
        data_path,
        target_size=img_shape,
        batch_size=c.batch_size,
        classes=c.labels,
        color_mode='rgb',
        class_mode='binary',
        shuffle=True,
        follow_links=True,
        seed=42)

- define callbacks for training

In [ ]:
# creating checkpoints during traing
def checkpoint_callback(model_path):
    checkpoint_dir = os.path.join(model_path, "checkpoints")
    checkpoint_name = os.path.join(
        checkpoint_dir, "fit-gen_epoch-{epoch:02d}_loss-{val_loss:.2f}.hdf5")

    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    return ModelCheckpoint(
        checkpoint_name, monitor='val_loss', save_best_only=False)


# logging training stuff
def tensorboard_callback(model_path):
    log_dir = os.path.join(model_path, "logs")

    if not os.path.exists(log_dir):
        os.makedirs(log_dir)

    return TensorBoard(
        log_dir=log_dir, histogram_freq=0,
        write_graph=True, write_images=True, update_freq="batch")


# create LR plots and log them
def save_lr_plots(model_path, history): 
    plot_path = os.path.join(model_path, "logs", "plots")
    if not os.path.exists(plot_path): 
        os.makedirs(plot_path)

    acc   = history['accuracy']
    loss  = history['loss']
    f1    = history['f1']
    lr    = history['lr']

    figure = plt.figure(figsize=(10,10))
    # plt.plot(lr, vac, 'k-', label='Validation Accuracy', linewidth=0.5)
    plt.plot(lr, acc, '-', label='Train Accuracy')
    plt.plot(lr, f1, 'k--', label='F1 Score', linewidth=0.5)
    plt.plot(lr, loss, '--', label='Train Loss')

    plt.xticks(ticks=lr)
    plt.setp(plt.gca().get_xticklabels(), rotation=45, ha="right")
    plt.ylabel('')
    plt.xlabel('Learning Rate')
    plt.legend()

    file_writer = tf.summary.create_file_writer(plot_path)
    with file_writer.as_default():
        tf.summary.image("LR range test", plot_to_image(figure), step=0)


def plot_to_image(figure):
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300)
    plt.close(figure)
    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)
    return image


# early stopping to prevent unneeded overfitting
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, mode='min', verbose=1, patience=20, restore_best_weights=True)

- model fit procedure

In [ ]:
def model_fit(model, train, val, model_path, class_weight={0: 1, 1: 1}, max_lr, lr_test=False):
    tensorboard = tensorboard_callback(model_path=model_path)
    tensorboard.set_model(model)
    checkpoint = checkpoint_callback(model_path=model_path)

    epochs = c.epochs
    steps_epoch = train.n / c.batch_size
    if lr_test:
        max_lr = 1
        epochs = c.epochs_lr
        step_size = int(train.n / c.batch_size * epochs)
    else: step_size = int(steps_epoch * 5)
    clr = CyclicLR(base_lr=0.0001, max_lr=max_lr,
                   step_size=step_size, mode='triangular')
    
    model.compile(optimizer='SGD',
                loss=binary_crossentropy,
                metrics=['accuracy', recall, precision, f1])
    
    history = model.fit_generator(
                generator=train,
                validation_data=val,
                epochs=epochs,
                class_weight=class_weight,
                callbacks=[clr, tensorboard, checkpoint, early_stop],
                shuffle=True,
                verbose=True
            ).history
    
    if lr_test: save_lr_plots(model_path, history)

## LEARNING RATE RANGE TEST

In [ ]:
for phi in [-5]:
    for dropout in np.linspace(0.1, 0.15, 3):
        default_name = f"{phi}-LR-dropout_{dropout}"

        model = efn.build_model(phi=phi, dropout=dropout)
        model.summary()

        input_shape = model.input_shape[1:3]

        train = load_data(input_shape, c.train_dir)
        train_up = load_data(input_shape, c.train_dir_up)

        val = load_data(input_shape, c.val_dir)

        # default
        model = efn.build_model(phi=phi, dropout=dropout)
        model_path = os.path.join(c.output_dir_models, default_name + "-default")
        model_fit(model, train, val, model_path, max_lr=1.0, lr_test=True)
        del model

        # weighted loss
        agrs = len(glob.glob(f'{c.train_dir}/agriculture/*.jpg'))
        other = len(glob.glob(f'{c.train_dir}/other/*.jpg'))
        class_weight = {0: 1.0, 
                        1: 1/ (agrs / (other+agrs))}

        model = efn.build_model(phi=phi, dropout=dropout)
        model_path = os.path.join(c.output_dir_models, default_name + "-weighted")
        model_fit(model, train, val, model_path, class_weight, max_lr=1.0, lr_test=True)
        del model

        # upsampled
        model = efn.build_model(phi=phi, dropout=dropout)
        model_path = os.path.join(c.output_dir_models, default_name + "-up")
        model_fit(model, train_up, val, model_path, max_lr=1.0, lr_test=True)
        del model

## TRAIN WITH SET MAX LR

In [ ]:
for phi in [-5]:
    for dropout in np.linspace(0.1, 0.15, 3):
        default_name = f"{phi}-dropout_{dropout}"

        model = efn.build_model(phi=phi, dropout=dropout)
        model.summary()

        input_shape = model.input_shape[1:3]

        train = load_data(input_shape, c.train_dir)
        train_up = load_data(input_shape, c.train_dir_up)

        val = load_data(input_shape, c.val_dir)

        # default
        max_lr = 0.1669
        
        model = efn.build_model(phi=phi, dropout=dropout)
        model_path = os.path.join(c.output_dir, default_name + "-default")
        model_fit(model, train, val, model_path, max_lr=max_lr)
        del model
        
        # weighted loss
        agrs = len(glob.glob(f'{c.train_dir}/agriculture/*.jpg'))
        other = len(glob.glob(f'{c.train_dir}/other/*.jpg'))
        class_weight = {0: 1.0, 
                        1: 1/ (agrs / (other+agrs))}
        
        max_lr = 0.2002

        model = efn.build_model(phi=phi, dropout=dropout)
        model_path = os.path.join(c.output_dir, default_name + "-weighted")
        model_fit(model, train, val, model_path, class_weight, max_lr=max_lr)
        del model

        # upsampled
        max_lr = 0.2673
        
        model = efn.build_model(phi=phi, dropout=dropout)
        model_path = os.path.join(c.output_dir, default_name + "-up")
        model_fit(model, train_up, val, model_path, max_lr=max_lr)
        del model

# Test lower $\phi$s

## Range Test

In [ ]:
dropouts = np.around(np.linspace(0.05, 0.2, 15, endpoint=False), 2)

for phi in np.arange(-15, -5):
    dropout = dropouts[phi]
    default_name = f"{phi}-LR-dropout_{dropout}"

    model = efn.build_model(phi=phi, dropout=dropout)
    model.summary()

    input_shape = model.input_shape[1:3]

    train = load_data(input_shape, c.train_dir)
    val = load_data(input_shape, c.val_dir)

    # weighted loss
    agrs = len(glob.glob(f'{c.train_dir}/agriculture/*.jpg'))
    other = len(glob.glob(f'{c.train_dir}/other/*.jpg'))
    class_weight = {0: 1.0, 
                    1: 1/ (agrs / (other+agrs))}

    model = efn.build_model(phi=phi, dropout=dropout)
    model_path = os.path.join(c.output_dir_models, default_name + "-weighted")
    model_fit(model, train, val, model_path, class_weight, max_lr=1.0, lr_test=True)
    del model

## Train full

In [ ]:
dropouts = np.around(np.linspace(0.05, 0.2, 15, endpoint=False), 2)

for phi in np.arange(-15, -5):
    dropout = dropouts[phi]
    default_name = f"{phi}-dropout_{dropout}"
    model = efn.build_model(phi=phi, dropout=dropout)
    model.summary()

    input_shape = model.input_shape[1:3]

    train = load_data(input_shape, c.train_dir)
    val = load_data(input_shape, c.val_dir)

    # weighted loss
    agrs = len(glob.glob(f'{c.train_dir}/agriculture/*.jpg'))
    other = len(glob.glob(f'{c.train_dir}/other/*.jpg'))
    class_weight = {0: 1.0, 
                    1: 1/ (agrs / (other+agrs))}

    max_lr = 0.2002

    model = efn.build_model(phi=phi, dropout=dropout)
    model_path = os.path.join(c.output_dir, default_name + "-weighted")
    model_fit(model, train, val, model_path, class_weight, max_lr=max_lr)
    del model